## AuxTel test of EFD after UTC change

Craig Lage - 14-Oct-21

In [5]:
import sys, time, os, asyncio

from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import astropy.units as u
from lsst_efd_client import EfdClient

In [7]:
# Set Cerro Pachon location
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [8]:
# for tab completion to work in current notebook instance
%config IPCompleter.use_jedi = False

In [9]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG
# Make matplotlib less chatty
logging.getLogger("matplotlib").setLevel(logging.WARNING)

In [10]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.01 sec
Read 79 history items for RemoteEvent(ATDomeTrajectory, 0, heartbeat)
Read historical data in 0.11 sec
Read 3 history items for RemoteEvent(ATArchiver, 0, heartbeat)
Read historical data in 0.11 sec
Read 10 history items for RemoteEvent(ATHeaderService, 0, heartbeat)
Read historical data in 0.20 sec
Read 20 history items for RemoteEvent(ATSpectrograph, 0, heartbeat)
Read historical data in 0.38 sec
Read 100 history items for RemoteEvent(ATHexapod, 0, heartbeat)
Read historical data in 0.39 sec
Read 100 history items for RemoteEvent(ATPtg, 0, heartbeat)
Read historical data in 0.41 sec
timeAn

[[None, None, None, None, None, None, None], [None, None, None, None]]

atspectrographCorrectionCompleted DDS read queue is filling: 29 of 100 elements
atspectrographCorrectionCompleted DDS read queue is filling: 23 of 100 elements
hexapodCorrectionStarted DDS read queue is filling: 19 of 100 elements
logMessage DDS read queue is filling: 24 of 100 elements
heartbeat DDS read queue is full (100 elements); data may be lost
ccsCommandState DDS read queue is full (100 elements); data may be lost
summaryState DDS read queue is filling: 49 of 100 elements
azimuthState DDS read queue is full (100 elements); data may be lost
imageRetrievalForArchiving DDS read queue is full (100 elements); data may be lost
startReadout DDS read queue is full (100 elements); data may be lost
pointingOffsetSummary DDS read queue is full (100 elements); data may be lost
azimuthInPosition DDS read queue is full (100 elements); data may be lost
heartbeat DDS read queue is filling: 81 of 100 elements
imageInOODS DDS read queue is full (100 elements); data may be lost
startIntegration D

In [11]:
# enable components if required
await atcs.enable({"atdome": "current", "ataos": "current", "athexapod": "current"})
await latiss.enable({"atspectrograph": "current"})

Enabling all components
Gathering settings.
Received settings from users.: {'atdome': 'current', 'ataos': 'current', 'athexapod': 'current'}
Couldn't get settingVersions event. Using empty settings.
Complete settings for atmcs.
Complete settings for atptg.
Complete settings for atpneumatics.
Complete settings for atdometrajectory.
Settings versions: {'atdome': 'current', 'ataos': 'current', 'athexapod': 'current', 'atmcs': '                                                                                                                                                                                                                                                                ', 'atptg': '', 'atpneumatics': '                                                                                                                                                                                                                                                                ', 'atdometrajectory': ''}


RuntimeError: Failed to transition ['atcamera', 'atarchiver'] to <State.ENABLED: 2>.

In [12]:
# I'm not sure why this errored out, but everything shows as ENABLED, so I'm driving on.
# All components enabled 11:35AM project time

In [ ]:
# take event checking out the slew commands to test telescope only
# otherwise it'll wait for the dome before completing slew command
atcs.check.atdome = False
atcs.check.atdometrajectory = False

In [13]:
# Take 5 biases to make sure everything is working
# All looks OK
for i in range(5):
    await latiss.take_bias(1)
    await asyncio.sleep(2)

Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 29 of 100 elements
logMessage DDS read queue is filling: 13 of 100 elements
logMessage DDS read queue is filling: 10 of 100 elements
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 11 of 100 elements
logMessage DDS read queue is filling: 21 of 100 elements
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 20 of 100 elements
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
Generating group_id
imagetype: BIAS, skip TCS synchronization.
BIAS 0001 - 0001
logMessage DDS read queue is filling: 10 of 100 elements


In [14]:
# point telescope to desired starting position
start_az=0
start_el=70
start_rot_pa=0
await atcs.point_azel(start_az, start_el, rot_tel=start_rot_pa, wait_dome=False)

Sending command
Stop tracking.
Tracking state: <AtMountState.STOPPING: 10>
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
allAxesInPosition python read queue is filling: 33 of 100 elements
Got False
Telescope not in position
Got True
Waiting for telescope to settle.
Dome following disabled. Ignoring dome position.
ATDome in position.
[Telescope] delta Alt = -009.987 deg; delta Az= -000.003 deg; delta N1 = -000.000 deg; delta N2 = +000.000 deg 
Telescope in position.


In [17]:
#declare offset sizes for tests

# For small slew and settle
d_az= 5.0 # degrees
d_el = 2.0 # degrees
d_rot= 0 # degrees

In [18]:
# get RA/DEC of current telescope Alt/Az position
az = Angle(start_az, unit=u.deg)
el = Angle(start_el, unit=u.deg)
print(f'orig az {az} and el {el}')
time_data = await atcs.rem.atptg.tel_timeAndDate.next(flush=True, timeout=2)
# This should be TAI and not UTC... so will be 37s off system clock seconds ??
curr_time_atptg = Time(time_data.mjd, format="mjd")
coord_frame_AltAz = AltAz(location=location, obstime=curr_time_atptg)
coord_frame_radec = ICRS()
coord_azel = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec = coord_azel.transform_to(coord_frame_radec)
print('Current Position is: \n {}'.format(coord_azel))
print('Current Position is: \n {}'.format(ra_dec))

# get RA/DEC of target position
az = Angle(start_az+d_az, unit=u.deg)
el = Angle(start_el+d_el, unit=u.deg)
print(f'target az {az} and el {el}')
coord_azel_target = AltAz(az=az, alt=el, location=location, obstime=curr_time_atptg)
ra_dec_target = coord_azel_target.transform_to(coord_frame_radec)
print('Target Position is: \n {}'.format(coord_azel_target))
print('Target Position is: \n {}'.format(ra_dec_target))

orig az 0.0 deg and el 70.0 deg
Current Position is: 
 <AltAz Coordinate (obstime=59501.780400171345, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (0., 70.)>
Current Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (233.43732671, -10.17428466)>
target az 5.0 deg and el 72.0 deg
Target Position is: 
 <AltAz Coordinate (obstime=59501.780400171345, location=(1819093.56876225, -5208411.6827961, -3195180.61110659) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (5., 72.)>
Target Position is: 
 <ICRS Coordinate: (ra, dec) in deg
    (235.01289864, -12.2363705)>


In [19]:
# Slew to starting position and take an image to check headers
# Image 006
await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image to get some header data
await latiss.take_engtest(exptime=2)


Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
Dome following disabled. Ignoring dome position.
ATDome in position.
Got False
Telescope not in position
[Telescope] delta Alt = +000.016 deg; delta Az = +000.329 deg; delta N1 = -000.000 deg; delta N2 = +068.473 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = -000.000 deg; delta Az = -000.016 deg; delta N1 = -000.000 deg; delta N2 = +064.625 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = -000.000 deg; delta Az = -000.011 deg; delta N1 = +000.000 deg; delta N2 = +058.650 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = -000.000 deg; delta Az = -000.012 deg; delta N1 = +000.000 deg; delta N2 = +054.679 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = +0

array([2021101400006])

logMessage DDS read queue is filling: 18 of 100 elements


In [21]:
# Slew to target position and take an image to check headers
# Image 007
await atcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec), rot=0.0,
                      slew_timeout=240., stop_before_slew=False, wait_settle=False)


print('track for 2s')
await asyncio.sleep(2)
# take a quick image to get some header data
await latiss.take_engtest(exptime=2)


Auto sky angle: 0.0 deg
Sending command
[Telescope] delta Alt = +000.000 deg; delta Az = -000.013 deg; delta N1 = -000.000 deg; delta N2 = -000.014 deg [Dome] delta Az = +003.040 deg
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
Dome following disabled. Ignoring dome position.
ATDome in position.
Got False
Telescope not in position
Got False
Telescope not in position
[Telescope] delta Alt = +001.971 deg; delta Az = +004.767 deg; delta N1 = +000.000 deg; delta N2 = -001.548 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = +001.293 deg; delta Az = +004.768 deg; delta N1 = -000.000 deg; delta N2 = -000.591 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = +000.005 deg; delta Az = +003.162 deg; delta N1 = -000.000 deg; delta N2 = -000.013 deg [Dome] delta Az = +00

array([2021101400007])

logMessage DDS read queue is filling: 26 of 100 elements


In [22]:
# This is where the tests are carried out.
# Got stuck the first time.  Backing up and re-running starting with ra_dec_target
# Images 008-017
for i in range(5):

    # Slew to starting position
    await atcs.slew_icrs(ra=str(ra_dec.ra), dec=str(ra_dec.dec), rot=0.0,
                          slew_timeout=240., stop_before_slew=False, wait_settle=False)


    print('track for 2s')
    await asyncio.sleep(2)
    # take an image
    await latiss.take_engtest(exptime=2)

    # Now slew to new target
    print('Starting to Slew to target')
    start_time = Time(Time.now(), format='fits')
    await atcs.slew_icrs(ra=str(ra_dec_target.ra), dec=str(ra_dec_target.dec),
                          rot=d_rot, slew_timeout=240., 
                          stop_before_slew=False, wait_settle=False)
    end_time = Time(Time.now(), format='fits')
    slew_time = TimeDelta(end_time - start_time, format='sec').value
    print(f'Time to slew is {slew_time} seconds')
    await latiss.take_engtest(exptime=2)
    await asyncio.sleep(2)


Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
Dome following disabled. Ignoring dome position.
ATDome in position.
Got False
Telescope not in position
[Telescope] delta Alt = -001.896 deg; delta Az = -004.738 deg; delta N1 = +000.000 deg; delta N2 = +001.983 deg [Dome] delta Az = +003.040 deg
[Telescope] delta Alt = -000.255 deg; delta Az = -004.209 deg; delta N1 = -000.000 deg; delta N2 = -000.011 deg [Dome] delta Az = +003.040 deg
Got True
Telescope in position.
track for 2s
Generating group_id
imagetype: ENGTEST, skip TCS synchronization.
ENGTEST 0001 - 0001
Starting to Slew to target
Auto sky angle: 0.0 deg
Sending command
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <St

In [23]:
# For shutdown of system
await atcs.stop_tracking()

Stop tracking.
Tracking state: <AtMountState.TRACKINGENABLED: 9>
Tracking state: <AtMountState.STOPPING: 10>
In Position: True.


In [24]:
# Putting everything back in standby.
await atcs.shutdown()

Disabling ATAOS corrections
Disabling ATAOS corrections.
Closing M1 cover vent gates.
Cover state <MirrorCoverState.CLOSED: 6>
M1 cover already closed.
M1 vent state <VentsPosition.CLOSED: 1>
M1 vents already closed.
Close dome.
ATDome Shutter Door is already closed. Ignoring.
Slew dome to Park position.
Disable dome trajectory following.
process as completed...
Dome following disabled. Ignoring dome position.
ATDome in position.
[Dome] delta Az = +003.040 deg
Axes in position.
Disable dome trajectory following.
Slew telescope to Park position.
Sending command
Stop tracking.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
Dome following disabled. Ignoring dome position.
ATDome in position.
[Telescope] delta Alt = +007.891 deg; delta Az= +003.601 deg; delta N1 = -000.000 deg; delta N2 = -074.942 deg 
[Telescope] delta Alt = +005.505 deg; delta 

In [25]:
await latiss.standby()

Unable to transition atcamera to <State.STANDBY: 5> NoneType: None
.
Traceback (most recent call last):
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/csc_utils.py", line 142, in set_summary_state
    state_data = await remote.evt_summaryState.aget(timeout=timeout)  # type: ignore
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/site-packages/lsst/ts/salobj/topics/read_topic.py", line 517, in aget
    await asyncio.wait_for(self._next_task, timeout=timeout)
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/python3.8/asyncio/tasks.py", line 501, in wait_for
    raise exceptions.TimeoutError()
asyncio.exceptions.TimeoutError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-0.7.0/lib/pytho

RuntimeError: Failed to transition ['atcamera', 'atarchiver'] to <State.STANDBY: 5>.

In [30]:
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.STANDBY)

RuntimeError: Cannot get summaryState from ATCamera

In [ ]:
# This is just stuff that might be needed to recover.

In [ ]:
await atcs.rem.atdome.cmd_start.set_start(settingsToApply="test", timeout=30)

In [ ]:
await salobj.set_summary_state(atcs.rem.atdome, salobj.State.STANDBY, settingsToApply="test")

In [ ]:
await salobj.set_summary_state(atcs.rem.atdometrajectory, salobj.State.ENABLED)

In [ ]:
await salobj.set_summary_state(latiss.rem.atspectrograph, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atcamera, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atheaderservice, salobj.State.STANDBY)
await salobj.set_summary_state(latiss.rem.atarchiver, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

In [ ]:
await salobj.set_summary_state(atcs.rem.atmcs, salobj.State.STANDBY)